# LLevar datos de torres a zonas de transporte

#### Definir en los parámetros de entrada si deben seleccionarse todos los ciudadanos o solo los estables

-----
**Table of contents**<a id='toc0_'></a>    
- [General](#toc1_1_)    
- [Funciones](#toc1_2_)    
- [_Main_](#toc1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
----

## <a id='toc1_1_'></a>[General](#toc0_)

In [11]:
import pickle
import os
import numpy as np
import pandas as pd
import itertools as it
from tqdm import tqdm as tqdm
import itertools as it

general_directory=os.getcwd()+'\\Data\\'

cases=('allt','hw','mo','no')
st=('st','unst')
days=('all','week')
allc=(('allt','st','week'),('allt','unst','week'),('allt','unst','all'),('hw','st','week'),('mo','st','week'),('mo','unst','all'),('no','st','week'),('no','unst','all'))


----
## <a id='toc1_2_'></a>[Funciones](#toc0_)

In [12]:
def get_days():
    if not os.path.isdir(general_directory): os.mkdir('Data')
    if not os.path.isdir(general_directory+'Days'): os.mkdir(general_directory+'Days')
    days=[i for i in os.listdir(general_directory+'Days') if i[-3:]=='dat']
    days.sort()
    if sys.argv[1]=='week':
        days=[days[i][0:10] for i in range(len(days)) if all([(i-j)%7 != 0 for j in [3,4]])]
    return days

In [13]:
def save_dataP(name,dictionary,location=general_directory):

    with open(location+'Saves/'+ name+'.pkl', 'wb') as tf:
        pickle.dump(dictionary,tf)

In [14]:
def load_hist(day,case,st):
    with open(general_directory+'Saves/'+day[6:10]+'_'+case+'_'+st+'_hist.pkl','r+b') as pk:
        hist=pickle.load(pk)
    return hist


def load_mat(day,case,st):
    m=np.genfromtxt(general_directory+'Saves/'+day[6:10]+'_'+case+'_'+st+'_mat.csv',delimiter=',')
    return m


In [15]:
def mean(case,st):
    Mm=0
    Mh=0
    with tqdm(desc="Mean of cases "+case,total=2*len(days)) as pbar:
        for day in days:
            Mm+=load_mat(day,case,st)
            pbar.update()
        np.savetxt(general_directory+'/Saves/Meanm_'+case+'_'+st+'_'+sys.argv[1]+'.csv',Mm/N,delimiter=',')
        for day in days:
            Mh+=load_hist(day,case,st)
            pbar.update()
        save_dataP('Meanh_'+case+'_'+st+'_'+sys.argv[1],Mh/N)

In [16]:

def load_mat(case,st,days):
    m=np.genfromtxt(general_directory+'Saves\\Meanm_'+case+'_'+st+'_'+days+'.csv',delimiter=',')
    return m


In [17]:
def matrixxz(case,st,days):
    matrix_t=load_mat(case,st,days)
    matrix_z=np.zeros((134,134))
    matrix_t=pd.DataFrame(matrix_t,index=towers,columns=towers)
    for comb in list(it.product(range(1,135),repeat=2)):
        erow=towers_prop[towers_prop['category']==comb[0]]
        ecol=towers_prop[towers_prop['category']==comb[1]]
        for r in erow['id']:
            for c in ecol['id']:
                matrix_z[comb[0]-1][comb[1]-1]+=matrix_t[r][c]*list(erow[towers_prop['id']==r]['percent'])[0]*list(ecol[towers_prop['id']==c]['percent'])[0]/10000
    print(sum(sum(matrix_z)))
    np.savetxt(general_directory+'Saves\\ZMeanm_'+case+'_'+st+'_'+days+'.csv',matrix_z,delimiter=',')
    return matrix_z

In [18]:
def matrix_z(case,st,days):
    matrix_t=load_mat(case,st,days)
    matrixz=np.zeros((134,134))
    with tqdm(desc="Charging zones "+case+'_'+st+'_'+days,total=pow(len(towers),2)) as pbar:
        for i in range(len(towers)):
            for j in range(len(towers)):
                erow=towers_prop[towers_prop['id']==towers[i]]
                ecol=towers_prop[towers_prop['id']==towers[j]]
                for r in range(len(erow['category'])):
                    for c in range(len(ecol['category'])):
                        matrixz[int(erow['category'].iloc[r])-1][int(ecol['category'].iloc[c])-1]+=matrix_t[i][j]*erow['percent'].iloc[r]*ecol['percent'].iloc[c]/10000
                pbar.update()
    np.savetxt(general_directory+'Saves\\ZMeanm_'+case+'_'+st+'_'+days+'.csv',matrixz,delimiter=',')
    print('Encontrados '+str(sum(sum(matrixz)))+ 'y la original '+ str(sum(sum(pd.DataFrame(load_mat('allt','st','all'),index=towers,columns=towers).drop(towers[-4:]).drop(towers[-4:],axis=1).to_numpy())) ) )
    

----
## <a id='toc1_3_'></a>[_Main_](#toc0_)

In [19]:
def main():
        days=get_days()
        try:
                with open(general_directory+'Saves/Towers.pkl','r+b') as pk:
                        towers=pickle.load(pk)
        except FileNotFoundError:
                print('You need the file Towers.pkl')
        try: 
                towers_prop=pd.read_csv(general_directory+'\\zt_havana_tower.csv').drop_duplicates().drop_duplicates(subset=['id','category'],keep='first')
                for i in list(towers_prop['id'].unique()):
                        if 99.5>=sum(towers_prop[towers_prop['id']==i]['percent']) or sum(towers_prop[towers_prop['id']==i]['percent'])>=100.5:
                                towers_prop.drop(towers_prop[towers_prop['id']==i].index[-1],inplace=True)       
        except FileNotFoundError:
            print('You need the file zt_havana_tower.csv')

        for i in list(towers_prop['id'].unique()):
                if 99.5>=sum(towers_prop[towers_prop['id']==i]['percent']) or sum(towers_prop[towers_prop['id']==i]['percent'])>=100.5:
                        towers_prop.drop(towers_prop[towers_prop['id']==i].index[-1],inplace=True)

        for c in allc: 
            matrix_z(*c)
        print('All matrixes done')

In [8]:
main()

Charging zones allt_st_week:   4%|█▊                                             | 3630/96100 [00:18<07:54, 194.90it/s]


Charging zones:   0%|                                                                            | 0/8 [00:19<?, ?it/s]


KeyboardInterrupt: 